In [3]:
import pandas as pd

df = pd.read_csv(
    '/mnt/nvme4/Datasets/MIMIC-IV/physionet.org/files/mimiciv/3.0/hosp/patients.csv.gz',
    compression='gzip'
)
print(df.head())
print(df.columns.tolist())


   subject_id gender  anchor_age  anchor_year anchor_year_group         dod
0    10000032      F          52         2180       2014 - 2016  2180-09-09
1    10000048      F          23         2126       2008 - 2010         NaN
2    10000058      F          33         2168       2020 - 2022         NaN
3    10000068      F          19         2160       2008 - 2010         NaN
4    10000084      M          72         2160       2017 - 2019  2161-02-13
['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']


In [5]:
import pandas as pd

# 读取 patients 表的 subject_id 和 dod 两列
patients_df = pd.read_csv(
    '/mnt/nvme4/Datasets/MIMIC-IV/physionet.org/files/mimiciv/3.0/hosp/patients.csv.gz',
    compression='gzip',
    usecols=['subject_id', 'dod']
)

# 读取 mimiciv_icd9.feather 表的 subject_id, _id, text 三列
mimicicd9_df = pd.read_feather(
    '/mnt/nvme2/yyc/medical-coding/files/data/mimiciv_icd9/mimiciv_icd9.feather',
    columns=['subject_id', '_id', 'text']
)

# 合并并设置索引，然后新增 LABEL 列
merged_df = (
    patients_df
    .merge(mimicicd9_df, on='subject_id', how='inner')
    .assign(LABEL=lambda df: df['dod'].notna().astype(int))
    .set_index('subject_id')
)

print(merged_df.head())



                   dod       _id  \
subject_id                         
10000032    2180-09-09  22595853   
10000032    2180-09-09  22841357   
10000032    2180-09-09  29079034   
10000032    2180-09-09  25742920   
10000248           NaN  20600184   

                                                         text  LABEL  
subject_id                                                            
10000032    name unit no admission date discharge date dat...      1  
10000032    name unit no admission date discharge date dat...      1  
10000032    name unit no admission date discharge date dat...      1  
10000032    name unit no admission date discharge date dat...      1  
10000248    name unit no admission date discharge date dat...      0  


In [6]:
merged_df.rename(columns={'text': 'TEXT'}, inplace=True)
merged_df['LABEL'] = merged_df['dod'].notna().astype(int)

# 查看结果
merged_df.head()

,dod,_id,TEXT,LABEL
subject_id,,,,
10000032,2180-09-09,22595853,name unit no admission date discharge date dat...,1
10000032,2180-09-09,22841357,name unit no admission date discharge date dat...,1
10000032,2180-09-09,29079034,name unit no admission date discharge date dat...,1
10000032,2180-09-09,25742920,name unit no admission date discharge date dat...,1
10000248,NaN,20600184,name unit no admission date discharge date dat...,0


In [10]:
# 如果 subject_id 已是索引，重命名索引名称为 ID
merged_df.rename_axis('subject_id', inplace=True)

# 如果 subject_id 仍为列，则使用下面这行重命名列：
# merged_df.rename(columns={'subject_id': 'ID'}, inplace=True)

merged_df.head()

,dod,_id,TEXT,LABEL
subject_id,,,,
10000032,2180-09-09,22595853,name unit no admission date discharge date dat...,1
10000032,2180-09-09,22841357,name unit no admission date discharge date dat...,1
10000032,2180-09-09,29079034,name unit no admission date discharge date dat...,1
10000032,2180-09-09,25742920,name unit no admission date discharge date dat...,1
10000248,NaN,20600184,name unit no admission date discharge date dat...,0


In [8]:
merged_df.to_csv('merged_df.csv', index=True)

In [9]:
import pandas as pd

# 读取合并后的 CSV（根据你的工作目录调整路径）
df = pd.read_csv('merged_df.csv', index_col=0)

# 统计
total_samples = len(df)
label_counts = df['LABEL'].value_counts().to_dict()

print(f"Total samples: {total_samples}")
print(f"LABEL = 0: {label_counts.get(0, 0)}")
print(f"LABEL = 1: {label_counts.get(1, 0)}")

Total samples: 209297
LABEL = 0: 138081
LABEL = 1: 71216


In [11]:
import pandas as pd

# 1. 读取 admissions 表的 subject_id 和 hospital_expire_flag 两列
admissions = pd.read_csv(
    '/mnt/nvme4/Datasets/MIMIC-IV/physionet.org/files/mimiciv/3.0/hosp/admissions.csv.gz',
    compression='gzip',
    usecols=['subject_id', 'hospital_expire_flag']
)

# 2. 如果同一 subject_id 有多条住院记录，汇总为一个标签（只要有一次院内死亡便记为 1）
adm_flag = (
    admissions
    .groupby('subject_id', as_index=False)['hospital_expire_flag']
    .max()
)

# 3. 将 hospital_expire_flag 合并到 merged_df
merged_df = (
    merged_df
    .reset_index()  # 把 subject_id 从索引变回列
    .merge(adm_flag, on='subject_id', how='left')
    .set_index('subject_id')
)


In [12]:
merged_df.head()

,dod,_id,TEXT,LABEL,hospital_expire_flag
subject_id,,,,,
10000032,2180-09-09,22595853,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,22841357,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,29079034,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,25742920,name unit no admission date discharge date dat...,1,0
10000248,NaN,20600184,name unit no admission date discharge date dat...,0,0


In [13]:
merged_df['LABEL'] = (
    (merged_df['hospital_expire_flag'] == 0) &
    (merged_df['LABEL'] == 1)
).astype(int)

In [14]:
merged_df.head()

,dod,_id,TEXT,LABEL,hospital_expire_flag
subject_id,,,,,
10000032,2180-09-09,22595853,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,22841357,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,29079034,name unit no admission date discharge date dat...,1,0
10000032,2180-09-09,25742920,name unit no admission date discharge date dat...,1,0
10000248,NaN,20600184,name unit no admission date discharge date dat...,0,0


In [15]:
merged_df.to_csv('merged_df.csv', index=True)

In [16]:
import pandas as pd

# 读取合并后的 CSV（根据你的工作目录调整路径）
df = pd.read_csv('merged_df.csv', index_col=0)

# 统计
total_samples = len(df)
label_counts = df['LABEL'].value_counts().to_dict()

print(f"Total samples: {total_samples}")
print(f"LABEL = 0: {label_counts.get(0, 0)}")
print(f"LABEL = 1: {label_counts.get(1, 0)}")

Total samples: 209297
LABEL = 0: 160634
LABEL = 1: 48663


In [17]:
# 将 merged_df 随机抽取五分之一样本
merged_df = merged_df.sample(frac=0.2, random_state=42)


# merged_df.reset_index(inplace=True)

# 查看抽样后统计
total = len(merged_df)
counts = merged_df['LABEL'].value_counts().reindex([0, 1], fill_value=0)
print(f"Total samples: {total}")
print(f"LABEL = 0: {counts.loc[0]}")
print(f"LABEL = 1: {counts.loc[1]}")


Total samples: 41859
LABEL = 0: 32107
LABEL = 1: 9752


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 假设 merged_df 已经是抽样后的 DataFrame，索引为 'ID' 或 'subject_id'
# 首先将 merged_df 重置索引（如果需要），并打乱顺序
df = merged_df.reset_index().sample(frac=1, random_state=42)

# 先划分训练集 (80%) 和 临时集 (20%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['LABEL'])

# 再将临时集按 50/50 划分为 验证集 (10%) 和 测试集 (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['LABEL'])

# 保存为 CSV
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [19]:
print(f"Train samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


Train samples: 33487
Validation samples: 4186
Test samples: 4186


In [25]:
import pandas as pd

# 读取 CSV
df = pd.read_csv('/mnt/nvme2/yyc/clinicalbert/dead/data40k/test.csv')

# 重命名 LABEL 列为 Label
df.rename(columns={'subject_id': 'ID'}, inplace=True)

# 保存回原文件或另存
df.to_csv('/mnt/nvme2/yyc/clinicalbert/dead/data40k/test.csv', index=False)
